Ejercisio 10

In [82]:
import numpy as np

In [83]:
A = np.array(([[4,1,1],
              [1,3,2],
              [1,2,5]]))

In [84]:
np.linalg.eig(A)

EigResult(eigenvalues=array([6.89510652, 3.39729507, 1.70759841]), eigenvectors=array([[ 0.43170413,  0.88573564,  0.17059871],
       [ 0.49725362, -0.07589338, -0.86427949],
       [ 0.75257583, -0.45794385,  0.47319874]]))

In [85]:
def Rotacion (A, iter_max, tol = 1e-10):

  iter = 0
  n = len(A)
  V = np.eye(n)
  while iter < iter_max:
    iter += 1

    max_val = 0
    p, q = 0, 0
    for i in range(n):
        for j in range(i + 1, n):
            if abs(A[i][j]) > max_val:
                max_val = abs(A[i][j])
                p, q = i, j

    if max_val < tol:
            break
    if A[p][p] != A[q][q]:
      delta = 0.5 * np.arctan(2*A[p][q]/(A[p][p] - A[q][q]))
    else:
      delta = np.pi/4
    R = np.eye(n)
    R[p][p] = R[q][q] = np.cos(delta)
    R[p][q] = -np.sin(delta)
    R[q][p] = np.sin(delta)

    A = R.T @ A @ R
    V = V @ R
  A = np.diag(A)
  return A, V

In [86]:
R = Rotacion(A, 1000)
R

(array([3.39729507, 1.70759841, 6.89510652]),
 array([[ 0.88573564, -0.17059871,  0.43170413],
        [-0.07589338,  0.86427949,  0.49725362],
        [-0.45794385, -0.47319874,  0.75257583]]))

ejercisio 12

In [87]:
G1=(lambda x1,x2: np.log( x1**2 + x2**2 ) - np.sin(x1*x2), \
    lambda x1,x2: np.exp(x1-x2) + np.cos(x1*x2))

In [88]:
G2=(lambda x1,x2,x3: 6*x1 - 2*np.cos(x2*x3) - 1., \
    lambda x1,x2,x3: 9*x2 + np.sqrt( x1**2 + np.sin(x3) + 1.06 ) + 0.9, \
    lambda x1,x2,x3: 60*x3 + 3*np.exp(-x1*x2) + 10*np.pi - 3.)

In [89]:
def GetJacobian_1(f,r,h=1e-3):

  n = r.shape[0]

  J = np.zeros((n,n))

  for i in range(n):
    for j in range(n):

      rf = r.copy()
      rb = r.copy()

      rf[j] = rf[j] + h
      rb[j] = rb[j] - h

      J[i,j] = (f[i](rf[0],rf[1]) - f[i](rb[0],rb[1]))/(2*h)

  return J

In [90]:
def GetJacobian_2(f,r,h=1e-3):

  n = r.shape[0]

  J = np.zeros((n,n))

  for i in range(n):
    for j in range(n):

      rf = r.copy()
      rb = r.copy()

      rf[j] = rf[j] + h
      rb[j] = rb[j] - h

      J[i,j] = (f[i](rf[0],rf[1],rf[2]) - f[i](rb[0],rb[1],rb[2]))/(2*h)

  return J

In [107]:
Dx = lambda x1,x2,h=1e-5: (GetJacobian_1(x1+h,x2)-GetJacobian_1(x1-h,x2))/(2*h)
Dy = lambda x1,x2,h=1e-5: (GetJacobian_1(x1,x2+h)-GetJacobian_1(x1,x2-h))/(2*h)
Gradient_1 = lambda x,y: np.array([Dx(x,y),Dy(x,y)])

In [108]:
Dx = lambda x1,x2,x3,h=1e-5: (GetJacobian_2(x1+h,x2,x3)-GetJacobian_2(x1-h,x2,x3))/(2*h)
Dy = lambda x1,x2,x3,h=1e-5: (GetJacobian_2(x1,x2+h,x3)-GetJacobian_2(x1,x2-h,x3))/(2*h)
Dz = lambda x1,x2,x3,h=1e-5: (GetJacobian_2(x1,x2,x3+h)-GetJacobian_2(x1,x2,x3-h))/(2*h)
Gradient_2 = lambda x,y: np.array([Dx(x,y),Dy(x,y), Dz])

In [109]:
def NormaJ(J):
    return np.sqrt(np.sum(J**2))

In [110]:
def Minimizer_1(f, r0, N=1000, gamma=0.01):
    r = np.zeros((N, 2))
    r[0] = r0

    Norm = np.zeros(N)
    J = GetJacobian_1(f, r[0])
    Norm[0] = NormaJ(J)

    for i in range(1, N):
        J = GetJacobian_1(f, r[i-1])
        grad = np.zeros(2)

        for j in range(2):
            rf = r[i-1].copy()
            rf[j] += 1e-3
            J_forward = GetJacobian_1(f, rf)
            norm_forward = NormaJ(J_forward)

            rb = r[i-1].copy()
            rb[j] -= 1e-3
            J_backward = GetJacobian_1(f, rb)
            norm_backward = NormaJ(J_backward)

            grad[j] = (norm_forward - norm_backward) / (2 * 1e-3)


        r[i] = r[i-1] - gamma * grad


        J = GetJacobian_1(f, r[i])
        Norm[i] = NormaJ(J)


        if np.abs(Norm[i] - Norm[i-1]) < 1e-6:
            break

    return r[i]

In [111]:
Minimizer_1(G1, np.array([2,2]), N=1000, gamma=0.01)

array([2.2024868 , 2.68999517])

In [112]:
def Minimizer_2(f, r0, N=1000, gamma=0.01):
    r = np.zeros((N, 3))
    r[0] = r0

    Norm = np.zeros(N)
    J = GetJacobian_2(f, r[0])
    Norm[0] = NormaJ(J)

    for i in range(1, N):
        J = GetJacobian_2(f, r[i-1])
        grad = np.zeros(3)

        for j in range(3):
            rf = r[i-1].copy()
            rf[j] += 1e-3
            J_forward = GetJacobian_2(f, rf)
            norm_forward = NormaJ(J_forward)

            rb = r[i-1].copy()
            rb[j] -= 1e-3
            J_backward = GetJacobian_2(f, rb)
            norm_backward = NormaJ(J_backward)

            grad[j] = (norm_forward - norm_backward) / (2 * 1e-3)


        r[i] = r[i-1] - gamma * grad


        J = GetJacobian_2(f, r[i])
        Norm[i] = NormaJ(J)


        if np.abs(Norm[i] - Norm[i-1]) < 1e-6:
            break

    return r[i]

In [113]:
Minimizer_2(G2, np.array([0,0,0]), N=1000, gamma=0.01)

array([ 0.00000000e+00, -1.77635684e-13,  1.82468554e-05])

ejercisio 13

In [116]:
def Jacobiano_sigma4 (f,r,h=0.01):

    n = r.shape[0]
    J = np.zeros((n,n))

    for i in range(n):
      for j in range(n):

        r1 = r.copy()
        r2 = r.copy()
        r3 = r.copy()
        r4 = r.copy()

        r1[j] += 2*h
        r2[j] += h
        r3[j] -= h
        r4[j] -= 2*h
        J[i,j] = (-f[i](r1[0],r1[1],r1[2]) + 8*f[i](r2[0],r[1],r2[2]) - 8*f[i](r3[0],r3[1],r3[2]) + f[i](r4[0],r4[1],r4[2]))/(12*h)

    return J

In [119]:
def Jacobiano_sigma2(f,r,h=0.01):

  n = r.shape[0]

  J = np.zeros((n,n))

  for i in range(n):
    for j in range(n):

      rf = r.copy()
      rb = r.copy()

      rf[j] = rf[j] + h
      rb[j] = rb[j] - h

      J[i,j] = (f[i](rf[0],rf[1],rf[2]) - f[i](rb[0],rb[1],rb[2]))/(2*h)

  return J

In [123]:
a = Jacobiano_sigma4(G2, np.array([0,0,0]))
b = Jacobiano_sigma2(G2, np.array([0,0,0]))
c = Jacobiano_sigma2(G2, np.array([0,0,0]), h=0.01**2)

print(a-b)
print(b-c)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-7.40148683e-15 -7.40148683e-15 -7.40148683e-15]
 [-5.92118946e-14 -5.92118946e-14 -5.92118946e-14]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


ejercisio 14

In [126]:
import sympy as sp

J_x = sp.Matrix([[0, 0, 0],
                  [0, 0, -1],
                  [0, 1, 0]])

J_y = sp.Matrix([[0, 0, 1],
                  [0, 0, 0],
                  [-1, 0, 0]])

J_z = sp.Matrix([[0, -1, 0],
                  [1, 0, 0],
                  [0, 0, 0]])


def commutator(A, B):
    return A*B - B*A


commutator_xy = commutator(J_x, J_y)
commutator_xz = commutator(J_x, J_z)
commutator_yz = commutator(J_y, J_z)

Rcommutator_xy = np.zeros((3,3))
Rcommutator_xz = np.zeros((3,3))
Rcommutator_yz = np.zeros((3,3))


commutator_xy, commutator_xz, commutator_yz

(Matrix([
 [0, -1, 0],
 [1,  0, 0],
 [0,  0, 0]]),
 Matrix([
 [0, 0, -1],
 [0, 0,  0],
 [1, 0,  0]]),
 Matrix([
 [0, 0,  0],
 [0, 0, -1],
 [0, 1,  0]]),
 array([[ 0., -1.,  0.],
        [ 1.,  0.,  0.],
        [ 0.,  0.,  0.]]),
 array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]),
 array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]))

Es posible evidenciar que:

$
  [J_x,J_y] = J_z\\
  [J_y,J_z] = J_x\\
  [J_z,J_x] = J_y
$

Y lo anterior solo se cumple si el algebra de conjuntos esta dada por
$[J_i, J_j ] = ϵ_{ijk}J_k,$